In [1]:
# SETUP
from IPython.display import display
import pandas as pd
import os
from copy import deepcopy
import numpy as np
from math import isclose

os.chdir("..")
from psuu import psuu_find_next_grid

sweep = "gateway_viability_sweep_ag1_"
psuu_find_next_grid(sweep)

Add the following to model/config/params.py:

gateway_viability_sweep_ag2_ = build_params("Base")
gateway_viability_sweep_ag2_["session_token_bucket_coefficient"] = [212.5, 400]
gateway_viability_sweep_ag2_["gateway_fee_per_relay"] = [55.0, 100]
gateway_viability_sweep_ag2_["application_fee_per_relay"] = [55.0, 100]
gateway_viability_sweep_ag2_["gateway_minimum_stake"] = [150000000000.0, 200000000000.0]
gateway_viability_sweep_ag2_["minimum_application_stake"] = [15000000000.0, 20000000000.0]
gateway_viability_sweep_ag2_["application_max_number"] = [5, 20, 100]
gateway_viability_sweep_ag2_["relays_per_session_gamma_distribution_scale"] = [100000, 300000, 900000]
create_sweep("gateway_viability_sweep_ag2_",gateway_viability_sweep_ag2_,config_option_map_sweep)


Add the following to model/config/experiment.py:

for i in range(1, 2593):
    experimental_setups["gateway_viability_sweep_ag2_{}".format(i)] = {
        "config_option_state": "Base",
        "config_option_params": "gateway_vi

/Users/seanmcowen/opt/anaconda3/envs/BlockScience/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'minimum_stake_servicer': [15000000000.0],
 'minimum_stake_period_servicer': [10],
 'minimum_pause_time': [10],
 'max_chains_servicer': [15],
 'relays_to_tokens_multiplier': [161.29],
 'slash_fraction_downtime': [1e-06],
 'downtime_jail_duration': [3600000000000],
 'minimum_servicers_per_session': [1],
 'maximum_servicers_per_session': [5],
 'application_fee_per_relay': [55.0, 100],
 'minimum_application_stake': [15000000000.0, 20000000000.0],
 'app_burn_per_session': [0],
 'app_burn_per_relay': [0],
 'block_proposer_allocation': [0.05],
 'dao_allocation': [0.1],
 'servicer_allocation': [0.85],
 'stake_per_app_delegation': [15000000000.0],
 'gateway_fee_per_relay': [55.0, 100],
 'gateway_minimum_stake': [150000000000.0, 200000000000.0],
 'session_token_bucket_coefficient': [212.5, 400],
 'dao_fee_percentage': [0.1],
 'validator_fee_percentage': [0.9],
 'transaction_fee': [0.01],
 'min_bootstrap_gateway_fee_per_relay': [8.5e-07],
 'maturity_relay_charge': [1.971e-06],
 'gateway_bootstr

In [5]:
display(df_thresholds)

print(df_thresholds.mean())

param_session_token_bucket_coefficient  param_gateway_fee_per_relay  param_application_fee_per_relay  param_gateway_minimum_stake  param_minimum_application_stake
25                                      20.0                         20                               1.000000e+11                 1.000000e+10                       1
                                                                                                                                   2.000000e+10                       1
                                                                                                      2.000000e+11                 1.000000e+10                       1
                                                                                                                                   2.000000e+10                       1
                                                                     30                               1.000000e+11                 1.000000e+10                      

In [1]:
import pandas as pd
import os
from copy import deepcopy
import numpy as np
from math import isclose

os.chdir("..")
from adaptive_multigrid import give_data_individual_names, find_best_individual_name, build_individual_dict_from_param_grid, update_param_grid
from gvs_threshold_calc import calc_gvs_thresh_ineq_met
from model.config.params import *


fitness_func = calc_gvs_thresh_ineq_met
sweep = "gateway_viability_sweep_ag5_"
kpis = pd.read_csv("simulation_data/{}.csv".format(sweep), index_col=0)
exec("param_config = {}".format(sweep))
next_name = sweep[:-2] + str(int(sweep[-2]) + 1) + "_"

variable_params = ['session_token_bucket_coefficient', 'gateway_fee_per_relay', 'application_fee_per_relay', 'gateway_minimum_stake', 'minimum_application_stake']
control_params = ['application_max_number', 'relays_per_session_gamma_distribution_scale']

params_to_evolve = {}
for x in variable_params:
    params_to_evolve[x] = param_config[x]

params_for_setup_only = {}
for x in control_params:
    params_for_setup_only[x] = param_config[x]

initial_param_grid = params_to_evolve
other_params_to_sweep = params_for_setup_only




/Users/seanmcowen/opt/anaconda3/envs/BlockScience/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
new_param_grid = deepcopy(initial_param_grid)
new_best_name = ""
new_best_fitness = -np.inf
times_repeat_best_name = 0
times_repeat_best_fitness = 0

old_param_grid = deepcopy(new_param_grid)
old_best_name = new_best_name
old_best_fitness = new_best_fitness

individual_dict = build_individual_dict_from_param_grid(old_param_grid)
        
        

data = kpis
data = give_data_individual_names(data)

new_best_name, new_best_val = find_best_individual_name(
            data, fitness_func=fitness_func, name_biased_towards=old_best_name
        )

assert not (
            new_best_name is None
        ), "The best individual should be somewhere in this dictionary."
print(new_best_name)

new_best_individual = individual_dict[new_best_name]

if new_best_name == old_best_name:
    times_repeat_best_name = times_repeat_best_name + 1
else:
    times_repeat_best_name = 0

if isclose(new_best_fitness, old_best_fitness):
    times_repeat_best_fitness = times_repeat_best_fitness + 1
else:
    times_repeat_best_fitness = 0
    
new_param_grid = update_param_grid(
                old_param_grid=old_param_grid, best_individual=new_best_individual
            )

Building the individual dict.
Individual dict is {'stbc:189.0625_gfpr:21.875_afpr:25.625_gms:137500000000.0_ams:187500000000.0': {'session_token_bucket_coefficient': 189.0625, 'gateway_fee_per_relay': 21.875, 'application_fee_per_relay': 25.625, 'gateway_minimum_stake': 137500000000.0, 'minimum_application_stake': 187500000000.0}, 'stbc:189.0625_gfpr:21.875_afpr:25.625_gms:137500000000.0_ams:193750000000.0': {'session_token_bucket_coefficient': 189.0625, 'gateway_fee_per_relay': 21.875, 'application_fee_per_relay': 25.625, 'gateway_minimum_stake': 137500000000.0, 'minimum_application_stake': 193750000000.0}, 'stbc:189.0625_gfpr:21.875_afpr:25.625_gms:143750000000.0_ams:187500000000.0': {'session_token_bucket_coefficient': 189.0625, 'gateway_fee_per_relay': 21.875, 'application_fee_per_relay': 25.625, 'gateway_minimum_stake': 143750000000.0, 'minimum_application_stake': 187500000000.0}, 'stbc:189.0625_gfpr:21.875_afpr:25.625_gms:143750000000.0_ams:193750000000.0': {'session_token_bucket

We have added individual names for the various parameter combinations.
The DataFrame now looks like                                                     Net Minting Rate  \
individual_name                                                        
stbc:189.0625_gfpr:21.875_afpr:25.625_gms:13750...          0.000007   
stbc:189.0625_gfpr:21.875_afpr:25.625_gms:13750...          0.000007   
stbc:189.0625_gfpr:21.875_afpr:25.625_gms:13750...          0.000007   
stbc:189.0625_gfpr:21.875_afpr:25.625_gms:13750...          0.000006   
stbc:189.0625_gfpr:21.875_afpr:25.625_gms:13750...          0.000006   

                                                       KPI C  \
individual_name                                                
stbc:189.0625_gfpr:21.875_afpr:25.625_gms:13750...  0.212434   
stbc:189.0625_gfpr:21.875_afpr:25.625_gms:13750...  0.186075   
stbc:189.0625_gfpr:21.875_afpr:25.625_gms:13750...  0.157042   
stbc:189.0625_gfpr:21.875_afpr:25.625_gms:13750...  0.152674   
stbc:189.06

In [3]:
from psuu import build_next_param_config_code



Add the following to model/config/params.py:

gateway_viability_sweep_ag6_ = build_params("Base")
gateway_viability_sweep_ag6_["session_token_bucket_coefficient"] = [189.0625, 200.78125]
gateway_viability_sweep_ag6_["gateway_fee_per_relay"] = [21.875, 22.1875]
gateway_viability_sweep_ag6_["application_fee_per_relay"] = [25.625, 25.9375]
gateway_viability_sweep_ag6_["gateway_minimum_stake"] = [137500000000.0, 140625000000.0]
gateway_viability_sweep_ag6_["minimum_application_stake"] = [190625000000.0, 193750000000.0]
gateway_viability_sweep_ag6_["application_max_number"] = [5, 20, 100]
gateway_viability_sweep_ag6_["relays_per_session_gamma_distribution_scale"] = [100000, 300000, 900000]
create_sweep("gateway_viability_sweep_ag6_",gateway_viability_sweep_ag6_,config_option_map_sweep)


Add the following to model/config/experiment.py:

for i in range(1, 289):
    experimental_setups["gateway_viability_sweep_ag6_{}".format(i)] = {
        "config_option_state": "Base",
        "config_optio